In [1]:
import nemo.collections.asr as nemo_asr
import pandas as pd

/home/cybernet-adm/miniconda3/envs/nemo_kenlm/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
nemo_asr.__version__

'2.0.0rc1'

In [2]:
ckpt = '/home/cybernet-adm/quartznet_infer/checkpoint/QuartzNet-Model-EG.nemo'
model = nemo_asr.models.EncDecCTCModel.restore_from(ckpt)

[NeMo W 2024-07-14 11:00:18 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train_manifest_EGAR.json
    sample_rate: 16000
    labels:
    - ' '
    - ء
    - آ
    - أ
    - ؤ
    - إ
    - ئ
    - ا
    - ب
    - ة
    - ت
    - ث
    - ج
    - ح
    - خ
    - د
    - ذ
    - ر
    - ز
    - س
    - ش
    - ص
    - ض
    - ط
    - ظ
    - ع
    - غ
    - ف
    - ق
    - ك
    - ل
    - م
    - ن
    - ه
    - و
    - ى
    - ي
    - چ
    - ڤ
    batch_size: 32
    shuffle: true
    max_duration: 16.7
    pin_memory: true
    is_tarred: false
    num_workers: 8
    
[NeMo W 2024-07-14 11:00:18 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(

[NeMo I 2024-07-14 11:00:18 features:289] PADDING: 16
[NeMo I 2024-07-14 11:00:19 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /home/cybernet-adm/quartznet_infer/checkpoint/QuartzNet-Model-EG.nemo.


### Read your test data

In [26]:
test_df = pd.read_excel('golden_wer_more_than_1_ar_2_iter.xlsx')

### Create the df according to our standards

In [29]:
df = pd.DataFrame(columns=['audio_filepath', 'inference', 'golden_transcription', 'transcriber\'s name', 'lang', 'bad', 'not clear', 'name', 'operator or client'])

In [31]:
df['audio_filepath'] = test_df['audio_filepath']

In [32]:
df.head()

,audio_filepath,inference,golden_transcription,transcriber's name,lang,bad,not clear,name,operator or client
0,/home/cybernet-adm/dataset_arb/pool_1_2_ARB/16...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/home/cybernet-adm/dataset_arb/pool_6_ARB/d389...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/home/cybernet-adm/dataset_arb/pool_6_ARB/ae49...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/home/cybernet-adm/dataset_arb/pool_6_ARB/ba60...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/home/cybernet-adm/dataset_arb/pool_6_ARB/a1d1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import os

#LM binary 
#For more info on how to create it Read the README FILE @ https://github.com/Mahmoud-ghareeb/NeMo

LM = '/home/cybernet-adm/quartznet_infer/EG_AR_VO.binary'


beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(model.cfg.decoder.vocabulary),
    beam_width=13,
    alpha=1.0, beta=1.5,
    lm_path=LM,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)

In [34]:
import numpy as np
from torch.nn import functional as f


def transcribe(x):
    """Get inference with just the model
    inputs: x => dataframe row
    outputs: the transcription
    """
    return model.transcribe([f'{x["audio_filepath"]}'])[0]


def transcribe_with_kenlm(x):
    """Get inference with your kenlm model
    inputs: x => dataframe row
    outputs: the transcription
    """
    Logits_hyps = model.transcribe(
                [f'{x["audio_filepath"]}'], return_hypotheses=True
            )
            
    logits = Logits_hyps[0].alignments
    probs = f.softmax(logits, dim=1).unsqueeze(0)
    result = beam_search_lm.forward(log_probs = probs, log_probs_length=None)
      
    return result[0][0][1]

In [35]:
with_lm = df.copy()
without_lm = df.copy()

In [36]:
with_lm['inference'] = with_lm.apply(transcribe_with_kenlm, axis=1)
without_lm['inference'] = without_lm.apply(transcribe, axis=1)

[NeMo W 2024-07-10 13:58:03 audio_to_text_dataset:62] `labels` is explicitly provided to the data loader, and is different from the `labels` provided at the model level config.
    If this is incorrect, please set the dataloader's `labels` to None.
Transcribing: 100%|██████████| 1/1 [00:00<00:00, 15.98it/s]


[NeMo W 2024-07-10 13:58:03 audio_to_text_dataset:62] `labels` is explicitly provided to the data loader, and is different from the `labels` provided at the model level config.
    If this is incorrect, please set the dataloader's `labels` to None.
Transcribing: 100%|██████████| 1/1 [00:00<00:00, 16.59it/s]
[NeMo W 2024-07-10 13:58:03 audio_to_text_dataset:62] `labels` is explicitly provided to the data loader, and is different from the `labels` provided at the model level config.
    If this is incorrect, please set the dataloader's `labels` to None.
Transcribing: 100%|██████████| 1/1 [00:00<00:00, 16.39it/s]
[NeMo W 2024-07-10 13:58:04 audio_to_text_dataset:62] `labels` is explicitly provided to the data loader, and is different from the `labels` provided at the model level config.
    If this is incorrect, please set the dataloader's `labels` to None.
Transcribing: 100%|██████████| 1/1 [00:00<00:00, 16.47it/s]
[NeMo W 2024-07-10 13:58:04 audio_to_text_dataset:62] `labels` is explic

In [38]:
with_lm.head(1)

,audio_filepath,inference,golden_transcription,transcriber's name,lang,bad,not clear,name,operator or client
0,/home/cybernet-adm/dataset_arb/pool_1_2_ARB/16...,شارع الجمهوريه,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test_df.columns

Index(['id', 'audio_filepath', 'transcriber_transcription', 'text',
       'robot_transcription', 'pool', 'duration', 'lang', 'wer', 'length',
       'check_robot', 'check_transcribers', 'check_golden', 'match_with_robot',
       'status', 'golden_status', 'transcriber', 'superviser', 'sex',
       'timestamp_create', 'timestamp_transcribed', 'comments', 'g2_check'],
      dtype='object')

In [39]:
with_lm['golden_transcription'] = test_df['golden_transcription']
with_lm['lang'] = test_df['lang']

without_lm['golden_transcription'] = test_df['golden_transcription']
without_lm['lang'] = test_df['lang']

### caluclate the WER

In [40]:
import Levenshtein

def calcWer(s1, s2):
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    return Levenshtein.distance(''.join(w1), ''.join(w2)) / len(''.join(w2))

In [41]:
with_lm['WER'] = with_lm.apply(lambda x: calcWer(x['inference'], x['golden_transcription']), axis=1)
without_lm['WER'] = without_lm.apply(lambda x: calcWer(x['inference'], x['golden_transcription']), axis=1)

In [42]:
with_lm.to_excel('wer_more_than_1_2_with_lm.xlsx', index=False)
without_lm.to_excel('wer_more_than_1_2_without_lm.xlsx', index=False)